In [1]:
import json
from pathlib import Path
import os
import bops as bp

session_path = ''
if not session_path:
    session_path = bp.select_folder('Select session folder',bp.dir_simulations())
    


NameError: name 'bp' is not defined

In [6]:
print(bops_settings)

{'current_project_folder': 'C:\\Git\\research_data\\TorsionToolAllModels'}
